In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

Import libraries

In [4]:
from nn import nn, preprocess, io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import mean_squared_error

In [5]:
#Load data
pos = io.read_text_file("./data/rap1-lieb-positives.txt")
neg = io.read_fasta_file("./data/yeast-upstream-1k-negative.fa")
print(len(pos))
print(len(neg))

137
3163


In [7]:
len(pos[0]), len([neg[0]])

(17, 1)